In [22]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib notebook 
%matplotlib notebook
%matplotlib notebook

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## Loading and preparing label maps

In [4]:
with open('../../../wsod/metadata/ont_m18/class_names_all.pkl', 'rb') as fin:
    mid2name_all = pickle.load(fin)

In [5]:
#with open('../../../wsod/metadata/ont_m18/class_names_all.csv', 'w') as fout:
#    for key, val in mid2name_all.items():
#        fout.write(f'{key},"{val}"\n')

In [6]:
with open('../../results/det_results_concat_34b.pkl', 'rb') as fin:
    det_results_concat = pickle.load(fin)

In [7]:
with open('../../results/det_results_merged_34b.pkl', 'rb') as fin:
    det_results_merged = pickle.load(fin)

## Choosing examples for each class to visualize

In [8]:
label_to_img = {key: [] for key in mid2name_all}
for key, val in det_results_merged.items():
    for det in val:
        if det['score'] >= 0.1:
            label_to_img[det['label']].append(key)

In [9]:
all_labels = list(label_to_img.keys())

In [10]:
sort_idx = np.argsort([-len(label_to_img[label]) for label in all_labels])

In [11]:
for i in sort_idx:
    print(all_labels[i], mid2name_all[all_labels[i]], len(label_to_img[all_labels[i]]))

/m/01g317 PER (Person) 67828
/m/04yx4 PER (Man) 14395
/m/0k4j VEH.WheeledVehicle.Car (Car) 8318
/m/04hgtk PER (Human_head) 5051
/m/0bs7_0t COM (Electronic_device) 4843
/m/03jm5 FAC.Building.House (House) 3806
/m/0hf58v5 COM (Luggage_and_bags) 3519
/m/03bt1vf PER (Woman) 3290
/m/07yv9 VEH (Vehicle) 2098
/m/07k1x COM.Equipment (Tool) 1626
/m/03120 COM.Flag.Flag (Flag) 1075
/m/019jd VEH.Watercraft.Boat (Boat) 1048
/m/07r04 VEH.WheeledVehicle.Truck (Truck) 995
/m/07cx4 COM (Telephone) 987
/m/021sj1 FAC.Building.OfficeBuilding (Office_building) 917
/m/0cmf2 VEH.Aircraft.Airplane (Airplane) 672
/m/01prls VEH.WheeledVehicle (Land_vehicle) 616
/m/0cgh4 FAC.Building (Building) 453
/m/07cmd VEH.MilitaryVehicle.Tank (Tank) 453
/m/02p0zyj Conflict.Attack (Riot) 402
/m/01bjv VEH.WheeledVehicle.Bus (Bus) 336
/m/01bl7v PER (Boy) 315
/m/0199g VEH.WheeledVehicle (Bicycle) 306
/m/07jdr VEH.WheeledVehicle.Train (Train) 303
/m/01fdzj FAC.Structure.Tower (Tower) 293
/m/05r655 PER (Girl) 225
/m/02_41 Disast

In [12]:
select_labels = []
for l in all_labels:
    if mid2name_all[l].startswith('WEA'):
        select_labels.append(l)

In [13]:
select_labels

['/m/04ctx',
 '/m/083kb',
 '/m/04ylt',
 '/m/06nrc',
 '/m/06y5r',
 '/m/0gxl3',
 '/m/02gzp',
 '/m/020kz',
 '/m/06c54',
 '/m/0jb3',
 '/m/01vzwd',
 '/m/034qg',
 '/m/04zjc',
 '/m/02bb1s',
 '/m/065zrn9',
 '/m/032rk',
 '/m/0728h',
 '/m/0ct4f',
 '/m/05t_pq',
 '/m/06fk8',
 'syn_006',
 'syn_008',
 'syn_017']

In [14]:
#select_labels = ['/m/01fnck', '/m/019jd', '/m/03120', '/m/02_41', '/m/07cmd', '/m/06q40', '/m/03qtwd', '/m/09x0r', '/m/01xgg_', '/m/08qrwn', '/m/012n4x', '/m/02p16m6', '/m/02lbcq', '/m/09ct_', '/m/0g54v5d', '/m/0bg2p', '/m/0ct4f', '/m/01nd_n', '/m/09rvcxw', '/m/01nl4x', '/m/04ctx', '/m/04ylt', '/m/0gvss07', '/m/01rzcn', '/m/06nrc', '/m/0cyfs', '/m/0f5lx', '/m/04zjc', '/m/0dhz0', '/m/01lcw4', '/m/03htg', '/m/0gxl3', '/m/01bq8v', '/m/0lt4_', '/m/02gzp', '/m/02yjc', '/m/012n7d', '/m/0jb3', '/m/0dwx7']
select_labels = [all_labels[i] for i in sort_idx]

## Visualizing Results

In [15]:
mid2name_extended = {}
extended_classes = set()
for key in all_labels:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('voc','PA'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J'), ('voc/J','PA/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_all[key] + f' ({abv})'
extended_classes = list(extended_classes)
mid2idx_extended = {}
for i, key in enumerate(extended_classes):
    mid2idx_extended[key] = i
category_index_extended = {mid2idx_extended[key]:{'id': mid2idx_extended[key], 'name': mid2name_extended[key]} for key in mid2name_extended}

In [16]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [17]:
result_dict = {
    'Concatenated': det_results_concat,
    'Merged': det_results_merged,
}

In [18]:
with open('../../../../data/eval_m18/kf_id2path.pkl', 'rb') as fin:
    kf_id_to_img_path = pickle.load(fin)

In [19]:
model_abv_dict = {
    'coco': 'CO',
    'voc': 'PA',
    'oi': 'OI',
    'ws': 'WS',
    'coco/J': 'CO/J',
    'voc/J': 'PA/J',
    'oi/J': 'OI/J',
    'ws/J': 'WS/J',
}

In [20]:
def show(imgid, source, thresh):
    filename = kf_id_to_img_path[imgid]
    print(imgid)
    with open(filename, 'rb') as fin:
        if filename.endswith('.ldcc'):
            _ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    
    detections = result_dict[source][imgid]
    boxes = np.asarray([det['bbox_normalized'] for det in detections])[:,[1,0,3,2]]
    scores = [det['score'] for det in detections]
    label_idx = [mid2idx_extended[det['label'] + '/' + det['model']] for det in detections]
    
    # print([(mid2name_extended[extended_classes[item]], boxes[i]) for i, item in enumerate(label_idx) if scores[i] > thresh])
    
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      label_idx,
      scores,
      category_index_extended,
      use_normalized_coordinates=True,
      min_score_thresh=thresh-0.001,
      max_boxes_to_draw=1000,
      line_thickness=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()    

In [23]:
for label in select_labels[:10]:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    print(imgid)
    w = interactive(show, 
        imgid=fixed(imgid),
        source=widgets.RadioButtons(options=['Concatenated', 'Merged'], value='Merged'), 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
    )
    w.children[-1].layout.height = '600px'
    display(w)
    

PER (Person)
HC00029AA_855


interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…

PER (Man)
HC000TL2I_24


interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…

VEH.WheeledVehicle.Car (Car)
IC0019N35_150


...

interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…

VEH (Vehicle)
IC001L3AF_847


interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…

COM.Equipment (Tool)
IC0019N2P_80


interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…